# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qamdo,31.1667,97.2333,45.90,25,99,8.97,CN,1711884493
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,34.39,88,75,10.00,RU,1711884494
2,2,ushuaia,-54.8000,-68.3000,42.46,81,75,8.05,AR,1711884460
3,3,adamstown,-25.0660,-130.1015,73.87,64,98,32.23,PN,1711884494
4,4,olonkinbyen,70.9221,-8.7187,19.67,68,100,17.22,SJ,1711884494


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"],
    alpha = .5
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Humidity"] < 75) \
                    & (city_data_df["Max Temp"] > 65) \
                    & (city_data_df["Max Temp"] < 85) \
                    & (city_data_df["Cloudiness"] < 50)]



# Drop any rows with null values
ideal_city_data_clean = ideal_city_data_df.dropna()



# Display sample data
ideal_city_data_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,hancheng,32.5186,112.3522,70.68,64,31,7.67,CN,1711884508
72,72,constantia,44.1833,28.6500,74.08,58,22,10.58,RO,1711884245
79,79,jinchang,38.4953,102.1739,69.89,17,6,9.13,CN,1711884512
132,132,nova sintra,14.8667,-24.7167,66.15,73,16,15.82,CV,1711884524
148,148,zhangye,38.9342,100.4517,72.00,11,28,15.95,CN,1711884529


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_clean.copy()
hotel_df = hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis = 1)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
59,hancheng,32.5186,112.3522,64,CN,
72,constantia,44.1833,28.6500,58,RO,
79,jinchang,38.4953,102.1739,17,CN,
132,nova sintra,14.8667,-24.7167,73,CV,
148,zhangye,38.9342,100.4517,11,CN,
164,al majaridah,19.1236,41.9111,59,SA,
167,hateg,45.6167,22.9500,25,RO,
169,port augusta,-32.5000,137.7667,51,AU,
173,kingston,17.9970,-76.7936,74,JM,
180,ilha de mocambique,-15.0342,40.7358,69,MZ,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    #print(params)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

     # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    #Convert the API response to JSON format
    name_address = name_address.json()  

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hancheng - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Chérica
jinchang - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
al majaridah - nearest hotel: No hotel found
hateg - nearest hotel: Hotel Ferdinand
port augusta - nearest hotel: Hotel Commonwealth
kingston - nearest hotel: Aphrodite Hotel
ilha de mocambique - nearest hotel: Ruby Backpackers
lanzhou - nearest hotel: 西园宾馆
gargalianoi - nearest hotel: Messinian Horizons
mianwali - nearest hotel: Qaiser
popilnia - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
yulinshi - nearest hotel: 榆溪大酒店
chinde - nearest hotel: Kanyum cottage village
dakhla - nearest hotel: Hotel El Massira
chocen - nearest hotel: Hotel Peliny
trogir - nearest hotel: villa Lucica Trogir
lozova - nearest hotel: "Оскар"
axum - nearest hotel: Delina Hotel
alice town - nearest hotel: No hotel found
tres palos - n

,City,Lat,Lng,Humidity,Country,Hotel Name
59,hancheng,32.5186,112.3522,64,CN,No hotel found
72,constantia,44.1833,28.6500,58,RO,Hotel Chérica
79,jinchang,38.4953,102.1739,17,CN,No hotel found
132,nova sintra,14.8667,-24.7167,73,CV,Residência Ka Dencho
148,zhangye,38.9342,100.4517,11,CN,Zhangye Hotel 张掖饭店
164,al majaridah,19.1236,41.9111,59,SA,No hotel found
167,hateg,45.6167,22.9500,25,RO,Hotel Ferdinand
169,port augusta,-32.5000,137.7667,51,AU,Hotel Commonwealth
173,kingston,17.9970,-76.7936,74,JM,Aphrodite Hotel
180,ilha de mocambique,-15.0342,40.7358,69,MZ,Ruby Backpackers


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = .5
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)